In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [2]:
df_anime = pd.read_csv('../dataset/anime-dataset-2023.csv')
df_ratings = pd.read_csv('../dataset/users-score-2023.csv') 

In [5]:
# Atur Pandas untuk menampilkan semua kolom
pd.set_option('display.max_columns', None)

# Lihat 3 baris pertama dari DataFrame
df_anime.head(3)


,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,Premiered,Status,Producers,Licensors,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",spring 1998,Finished Airing,Bandai Visual,"Funimation, Bandai Entertainment",Sunrise,Original,24 min per ep,R - 17+ (violence & profanity),41.0,43,78525,914193.0,1771505,https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",UNKNOWN,Finished Airing,"Sunrise, Bandai Visual",Sony Pictures Entertainment,Bones,Original,1 hr 55 min,R - 17+ (violence & profanity),189.0,602,1448,206248.0,360978,https://cdn.myanimelist.net/images/anime/1439/...
2,6,Trigun,Trigun,トライガン,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",spring 1998,Finished Airing,Victor Entertainment,"Funimation, Geneon Entertainment USA",Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,328.0,246,15035,356739.0,727252,https://cdn.myanimelist.net/images/anime/7/203...


In [9]:

df_head = df_ratings.head(2)
df_tail = df_ratings.tail(2)

# Buat DataFrame dengan "..." sebagai placeholder
ellipsis = pd.DataFrame([['...'] * len(df_ratings.columns)], columns=df_ratings.columns)

# Gabungkan df_head, ellipsis, dan df_tail
df_combined = pd.concat([df_head, ellipsis, df_tail], ignore_index=True)

# Tampilkan hasilnya
df_combined.head(10)

,user_id,Username,anime_id,Anime Title,rating
0,1,Xinil,21,One Piece,9
1,1,Xinil,48,.hack//Sign,7
2,...,...,...,...,...
3,1291097,JuunanaSai,226,Elfen Lied,10
4,1291097,JuunanaSai,8425,Gosick,10


In [4]:
df_ratings.head()

,user_id,Username,anime_id,Anime Title,rating
0,1,Xinil,21,One Piece,9
1,1,Xinil,48,.hack//Sign,7
2,1,Xinil,320,A Kite,5
3,1,Xinil,49,Aa! Megami-sama!,8
4,1,Xinil,304,Aa! Megami-sama! Movie,8


### prepocessing dataset anime

In [5]:
# Preprocess anime data
kolom_tetap = ['anime_id','Name', 'English name','Studios','Genres', 'Image URL']
kolom_hapus = [kolom for kolom in df_anime.columns if kolom not in kolom_tetap]
df_anime = df_anime.drop(kolom_hapus, axis=1)
df_anime['content'] = df_anime['Name'].fillna('') + ' ' + df_anime['Studios'].fillna('') + ' ' + df_anime['Genres'].fillna('')

def prep(text):
    text = text.lower()
    text = re.sub('\W+', ' ', text)
    text = re.sub('\d+', ' ', text)
    words = text.split()
    stopwords_list = set(stopwords.words('english'))
    clean_words = [word for word in words if (word not in stopwords_list) and len(word) > 2]
    porter = PorterStemmer()
    clean_words = [porter.stem(word) for word in clean_words]
    return clean_words

df_anime['content_prep'] = df_anime['content'].apply(prep)

<>:9: SyntaxWarning: invalid escape sequence '\W'
<>:10: SyntaxWarning: invalid escape sequence '\d'
<>:9: SyntaxWarning: invalid escape sequence '\W'
<>:10: SyntaxWarning: invalid escape sequence '\d'
C:\Users\User\AppData\Local\Temp\ipykernel_15720\2324586097.py:9: SyntaxWarning: invalid escape sequence '\W'
  text = re.sub('\W+', ' ', text)
C:\Users\User\AppData\Local\Temp\ipykernel_15720\2324586097.py:10: SyntaxWarning: invalid escape sequence '\d'
  text = re.sub('\d+', ' ', text)


In [6]:
df_anime.head(3)

,anime_id,Name,English name,Genres,Studios,Image URL,content,content_prep
0,1,Cowboy Bebop,Cowboy Bebop,"Action, Award Winning, Sci-Fi",Sunrise,https://cdn.myanimelist.net/images/anime/4/196...,"Cowboy Bebop Sunrise Action, Award Winning, Sc...","[cowboy, bebop, sunris, action, award, win, sci]"
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,"Action, Sci-Fi",Bones,https://cdn.myanimelist.net/images/anime/1439/...,"Cowboy Bebop: Tengoku no Tobira Bones Action, ...","[cowboy, bebop, tengoku, tobira, bone, action,..."
2,6,Trigun,Trigun,"Action, Adventure, Sci-Fi",Madhouse,https://cdn.myanimelist.net/images/anime/7/203...,"Trigun Madhouse Action, Adventure, Sci-Fi","[trigun, madhous, action, adventur, sci]"


In [7]:
total_anime = df_anime.shape[0]

print(f"Total jumlah anime dalam dataset: {total_anime}")

Total jumlah anime dalam dataset: 24905


In [8]:
df_anime.content[1]

'Cowboy Bebop: Tengoku no Tobira Bones Action, Sci-Fi'

In [9]:
# hapus anime yang mengandung genre/kata tertentu
hapus_anime_genre_tertentu = ['UNKNOWN', 'Girls Love', 'Erotica', 'Boys Love', 'Ecchi', 'Hentai']

# fungsi memeriksa anime yang mengandung unwanted text
def contains_unwanted_genre(text):
    for genre in hapus_anime_genre_tertentu:
        if genre in text:
            return True
    return False

# Fmemfilter anime dengan kata tertentu
df_filtered_anime = df_anime[~df_anime['content'].apply(contains_unwanted_genre)]


In [10]:
df_filtered_anime.head(3)

,anime_id,Name,English name,Genres,Studios,Image URL,content,content_prep
0,1,Cowboy Bebop,Cowboy Bebop,"Action, Award Winning, Sci-Fi",Sunrise,https://cdn.myanimelist.net/images/anime/4/196...,"Cowboy Bebop Sunrise Action, Award Winning, Sc...","[cowboy, bebop, sunris, action, award, win, sci]"
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,"Action, Sci-Fi",Bones,https://cdn.myanimelist.net/images/anime/1439/...,"Cowboy Bebop: Tengoku no Tobira Bones Action, ...","[cowboy, bebop, tengoku, tobira, bone, action,..."
2,6,Trigun,Trigun,"Action, Adventure, Sci-Fi",Madhouse,https://cdn.myanimelist.net/images/anime/7/203...,"Trigun Madhouse Action, Adventure, Sci-Fi","[trigun, madhous, action, adventur, sci]"


In [11]:
total_anime = df_filtered_anime.shape[0]

print(f"Total jumlah anime dalam dataset: {total_anime}")

Total jumlah anime dalam dataset: 10934


In [17]:
# Simpan dataset hasil prepocessing
df_filtered_anime.to_csv('../dataset/df_anime_rated3k.csv', index=False)

### prepocessing dataset rating

In [4]:
total_rating= df_ratings.shape[0]

print(f"Total jumlah rating dataset: {total_rating}")

Total jumlah rating dataset: 24325191


In [5]:
# Menghitung jumlah user_id yang unik
unique_users = df_ratings['user_id'].nunique()

print(f"Jumlah pengguna unik dalam dataset: {unique_users}")

Jumlah pengguna unik dalam dataset: 270033


In [6]:
# Urutkan data berdasarkan user_id dalam urutan menaik
df_sorted = df_ratings.sort_values(by='user_id', ascending=True)

# Memilih 3000 pengguna unik pertama
unique_users = df_sorted['user_id'].drop_duplicates().head(3000)

# Filter data rating untuk hanya pengguna yang dipilih
filtered_ratings = df_sorted[df_sorted['user_id'].isin(unique_users)]


# Mengecek hasil
print(f"Shape of filtered_ratings: {filtered_ratings.shape}")
print(f"Number of unique users in filtered_ratings: {filtered_ratings['user_id'].nunique()}")

Shape of filtered_ratings: (424827, 5)
Number of unique users in filtered_ratings: 3000


In [7]:
print(f"ukran dataset users setelah prepocessing: {filtered_ratings.shape}")

ukran dataset users setelah prepocessing: (424827, 5)


In [8]:
filtered_ratings.head()

,user_id,Username,anime_id,Anime Title,rating
0,1,Xinil,21,One Piece,9
166,1,Xinil,322,Paradise Kiss,8
167,1,Xinil,161,Peace Maker Kurogane,8
168,1,Xinil,213,Pia Carrot e Youkoso!!,4
169,1,Xinil,214,Pia Carrot e Youkoso!! 2,3


In [19]:
filtered_ratings.to_csv('../dataset/df_ratings.csv', index=False)